In [22]:
import requests
import os
import gzip
import glob

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from slugify import slugify

ModuleNotFoundError: No module named 'tqdm.notebook.tqdm'; 'tqdm.notebook' is not a package

In [36]:
YEAR = 2023
START_AMOUNT = 420000  # PLN, roughly 100k EUR
MAX_DIV = 300
#  Both ranges are non-inclusive!
RANGES = [START_AMOUNT // i for i in range(1, MAX_DIV)] + [i for i in range(START_AMOUNT // MAX_DIV, 0, -100)] + [0]
len(RANGES)

314

In [5]:
# These break the export, probably because of non-windows-1250 characters
# The downloader queries around these sums and so these recipients are missing from the years
SPECIALS = {
    2015: [
        21016.97, # http://beneficjenciwpr.minrol.gov.pl/outrecords/view/12485559
        51755.63, # http://beneficjenciwpr.minrol.gov.pl/outrecords/view/13334673
    ],
    2016: [
        18805.66, # http://beneficjenciwpr.minrol.gov.pl/outrecords/view/16350294
    ],
    2017: [
        
    ]
}

In [6]:
BASE_URL = 'http://beneficjenciwpr.minrol.gov.pl/search/export/csv/'

In [7]:
params = {'year': YEAR, 'totalfrom': 420000, 'totalto': None}

def get_url(params):
    return BASE_URL + '/'.join('{k}:{v}'.format(k=k, v=str(v).replace(".", ","))
                               for k, v in params.items() if v is not None) + '/sort:total/direction:asc'


In [14]:
def download(year, amount, total_to):
    url = get_url({'year': year, 'totalfrom': amount, 'totalto': total_to})
    filename = 'data/{year}_{amount}_{total_to}.csv.gz'.format(year=year, amount=amount, total_to=total_to)
    if os.path.exists(filename):
        return
    print(filename, url)
    response = requests.get(url, stream=True)
    with gzip.open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)

                
def start(year):
    specials = SPECIALS.get(year, [])
    total_to = None
    for amount in RANGES:
        print(amount, total_to)
        specs = []
        if total_to is not None:
            specs = [s for s in specials if amount < s < total_to]
        if specs:
            download(year, amount, specs[0])
            download(year, specs[0], total_to)
        else:
            download(year, amount, total_to)
        total_to = amount+0.001

In [16]:
download(2023, 0, 1000)

data/2023_0_1000.csv.gz http://beneficjenciwpr.minrol.gov.pl/search/export/csv/year:2023/totalfrom:0/totalto:1000/sort:total/direction:asc


In [32]:
def get_year(year):
    for filename in (glob.glob('data/{}_*.csv.gz'.format(year))):
        print(filename)
        yield pd.read_csv(filename, compression='gzip', encoding='windows-1250', sep=';')

In [29]:
# with gzip.open('data/2016_11666_12000.001.csv.gz') as f:
#     print(f.read()[:10400].decode('windows-1250'))

In [33]:
def apply_fixes(df):
    df = df.rename(columns={
        'Imię': 'recipient_firstname',
        'Nazwisko': 'recipient_lastname',
        'Nazwa': 'recipient_name',
        'Gmina': 'recipient_location',
        'Kod pocztowy': 'recipient_postcode',
        'Rok': 'year'
    })
    df = df.drop(columns=['Suma'])
    df['recipient_firstname'] = df['recipient_firstname'].fillna('').apply(str)
    df['recipient_lastname'] = df['recipient_lastname'].fillna('').apply(str)
    df['recipient_name'] = df['recipient_name'].where(df['recipient_name'].notnull(),
                                                      df['recipient_firstname'] + ' ' + df['recipient_lastname'])

    anonymous = 'Małe gospodarstwo'
    df['recipient_name'] = df['recipient_name'].str.replace(anonymous, '').str.strip()
    df = df.drop(columns=['recipient_firstname', 'recipient_lastname'])
    df['recipient_id'] = df.apply(lambda x: '%s-%s-%s' % (slugify(x['recipient_name']), x['recipient_postcode'], x['year']), 1)
    return df

In [37]:
get_year(2023)

<generator object get_year at 0x12a098190>

In [38]:
df = pd.concat(get_year(YEAR))
df = apply_fixes(df)
df.head()

data/2023_0_1000.csv.gz


ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 2


In [39]:
len(df)

NameError: name 'df' is not defined

In [8]:
cols = 'recipient_name|recipient_location|recipient_id|recipient_postcode|year'.split('|')
scheme_cols = list(set(df.columns) - set(cols))

In [9]:
len(scheme_cols)

102

In [10]:
# df['Suma'] = pd.to_numeric(df['Suma'].str.replace(',', '.'))
for c in scheme_cols:
    df[c] = pd.to_numeric(df[c].str.replace(',', '.'))

In [11]:
# mismatch = np.isclose(df[scheme_cols].sum(1), df['Suma'] , atol=1)
# len(df) - mismatch.sum()

In [12]:
df_final = pd.melt(df, id_vars=cols,  var_name='scheme', value_name='amount', value_vars=scheme_cols)
df_final.head()

,recipient_name,recipient_location,recipient_id,recipient_postcode,year,scheme,amount
0,KOŁO ŁOWIECKIE PUSZCZA W BIAŁOWIEŻY,BIAŁOWIEŻA,kolo-lowieckie-puszcza-w-bialowiezy-17-230-2017,17-230,2017,III.4 Program dla sektora owocow i warzyw,0.0
1,"""PRZEDSIĘBIORSTWO PRODUKCYJNO-HANDLOWO-USŁUGOW...",BUK,przedsiebiorstwo-produkcyjno-handlowo-uslugowe...,64-320,2017,III.4 Program dla sektora owocow i warzyw,0.0
2,ROBERT NOWOROLNIK - GOSPODARSTWO ROLNE,JODŁOWNIK,robert-noworolnik-gospodarstwo-rolne-34-620-2017,34-620,2017,III.4 Program dla sektora owocow i warzyw,0.0
3,JUSTYNA ROZMIAREK GOSPODARSTWO ROLNE,KÓRNIK,justyna-rozmiarek-gospodarstwo-rolne-62-035-2017,62-035,2017,III.4 Program dla sektora owocow i warzyw,0.0
4,"ROLNICZO WYTWÓRCZY KOMBINAT SPÓŁDZIELCZY IM, A...",LEŻAJSK,rolniczo-wytworczy-kombinat-spoldzielczy-im-an...,37-305,2017,III.4 Program dla sektora owocow i warzyw,0.0


In [13]:
df_final = df_final[(df_final['amount'] != 0.0) & df_final['amount'].notnull()]
df_final['country'] = 'PL'
df_final['currency'] = 'PLN'
df_final.head()

,recipient_name,recipient_location,recipient_id,recipient_postcode,year,scheme,amount,country,currency
48336,PRZEDSZKOLE NR 1 W NOWEJ DĘBIE,NOWA DĘBA,przedszkole-nr-1-w-nowej-debie-39-460-2017,39-460,2017,III.4 Program dla sektora owocow i warzyw,1893.11,PL,PLN
50864,PRZEDSZKOLE MIEJSKIE Z ODDZIAŁEM SPECJALNYM I ...,MIŃSK MAZOWIECKI,przedszkole-miejskie-z-oddzialem-specjalnym-i-...,05-300,2017,III.4 Program dla sektora owocow i warzyw,1405.63,PL,PLN
52683,PRZEDSZKOLE NR 3,PŁOŃSK,przedszkole-nr-3-09-100-2017,09-100,2017,III.4 Program dla sektora owocow i warzyw,3675.68,PL,PLN
81305,"""PRZEDSZKOLE NR 9 IM, """"JASIA I MAŁGOSI""""""",WOŁOMIN,przedszkole-nr-9-im-jasia-i-malgosi-05-200-2017,05-200,2017,III.4 Program dla sektora owocow i warzyw,3445.94,PL,PLN
87817,"ZESPÓŁ SZKÓŁ LEŚNYCH IM, UNII EUROPEJSKIEJ W ...",RUCIANE-NIDA,zespol-szkol-lesnych-im-unii-europejskiej-w-ru...,12-220,2017,III.4 Program dla sektora owocow i warzyw,6034.09,PL,PLN


In [14]:
df_final.to_csv('pl_{}.csv.gz'.format(YEAR), compression='gzip', index=False)